# Doc2vec MultiClassifier Model

1. 데이터  
    * data_0116를 사용함  
    * 텍스트로 이루어진 열을 하나의 열로 합쳐주고, Stopwards를 제거함  

2. fitting  
    * STEP 1 : Major열을 기준으로 정합성을 테스트하고, 올바르지 못한것은 et로 예측할 것이므로 따로 빼두기 **(string_compare fitting)**   
    * STEP 2 : Major열과 description을 jaro_distance로 비교하여 유사도를 도출함 **(string_compare fitting)**  
    * STEP 3 : 전체 train 데이터를 기준으로 et_300으로 학습함 **(et fitting)**
           
3. Prediction  
    * STEP 1 : Similarity와 사전에 정한 threshold를 비교하여 높은 레코드에 대해서만 string_compare로 예측을 진행 **(string_compare predict)**
    * STEP 2 : 그렇지 못한 데이터에 대해서는 et_300으로 예측값을 도출함 **(et predict)**

# 라이브러리 import & 데이터 로딩 & 전처리

In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import warnings

RANDOM_STATE = 42
np.seed = 42
DATA_PATH = "../data_0116/"

warnings.filterwarnings(action='ignore')
PATH_2017 = DATA_PATH + "train/KNOW_2017.csv"
PATH_2018 = DATA_PATH + "train/KNOW_2018.csv"
PATH_2019 = DATA_PATH + "train/KNOW_2019.csv"
PATH_2020 = DATA_PATH + "train/KNOW_2020.csv"

paths = [PATH_2017, PATH_2018, PATH_2019, PATH_2020]

know_train = [pd.read_csv(path) for path in paths]

TEST_PATH_2017 = DATA_PATH + "test/KNOW_2017_test.csv"
TEST_PATH_2018 = DATA_PATH + "test/KNOW_2018_test.csv"
TEST_PATH_2019 = DATA_PATH + "test/KNOW_2019_test.csv"
TEST_PATH_2020 = DATA_PATH + "test/KNOW_2020_test.csv"

TEST_PATHs = [TEST_PATH_2017, TEST_PATH_2018, TEST_PATH_2019, TEST_PATH_2020]

know_test = [pd.read_csv(path) for path in TEST_PATHs]

# doc2vec을 사용할거라 txt열을 모두 하나로 모아줘야해요
text_info_cols = {"2017": ['sim_job','bef_job','able_job','major'],
                  "2018": ['sim_job','bef_job','able_job','major'],
                  "2019": ['bef_job','able_job','major'],
                  "2020": ['major'],}
years = ['2017','2018','2019','2020']

for i, year in enumerate(years):
    text_info_col = text_info_cols[year]
    for col in text_info_col:
        know_train[i]['text_response'] = know_train[i]['text_response'] + ' ' + know_train[i][col]
        know_test[i]['text_response'] = know_test[i]['text_response'] + ' ' + know_test[i][col]
    know_train[i].drop(text_info_col,axis=1, inplace=True)
    know_test[i].drop(text_info_col,axis=1, inplace=True)

# text_response 내 존재하는 stopwords들을 제거해줍시다
stopwords = ['없다','없음','0','모름','공란']
for i in tqdm(range(4)):
    for k, doc in enumerate(know_train[i]['text_response']):
        doc_list = doc.split(' ')
        new_list = [word for word in doc_list if word not in stopwords]
        new_string = ''
        for word in new_list:
            new_string += word
            new_string += ' '
        know_train[i].loc[k,'text_response'] = new_string[:-1]
    for k, doc in enumerate(know_test[i]['text_response']):
        doc_list = doc.split(' ')
        new_list = [word for word in doc_list if word not in stopwords]
        new_string = ''
        for word in new_list:
            new_string += word
            new_string += ' '
        know_test[i].loc[k,'text_response'] = new_string[:-1]

100%|██████████| 4/4 [00:29<00:00,  7.43s/it]


# Fitting & Prediction

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def doc2vec_train(know_train):
    '''
    
    모든 text열이 합쳐진 text_response에 대해, Doc2vec을 적용하고, tagging함.
    
    '''
    my_texts = know_train[['idx','text_response']]
    my_texts_and_tags = [(row.text_response, [str(row.idx)]) for row in my_texts.itertuples()]
    TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in my_texts_and_tags]
    model = Doc2Vec(TRAIN_documents, vector_size=1000, window=4, epochs=40, min_count=0, workers=4)
    
    return model

def doc2vec_pred(know_train, know_test, model, viz = False):
    '''
     
    test set의 text_response와 가장 유사한 text_response를 train에서 찾음.
    해당 레코드의 knowcode를 prediction으로 생각함.
    
    '''
    Test_documents = [doc for doc in know_test['text_response']]
    
    pred_list = []
    sim_list = []
    for text in Test_documents:
        text_split = text.split(' ')
        inferred_v = model.infer_vector(text_split)
        most_similar_docs = model.docvecs.most_similar(positive=[inferred_v], topn=1)
        
        top1_train_idx = int(most_similar_docs[0][0])
        top1_sim = round(float(most_similar_docs[0][1]),2)
        sim_list.append(top1_sim)
        
        pred = list(know_train.loc[know_train['idx']==top1_train_idx]['knowcode'])[0]
        pred_list.append(pred)
        
    return pred_list, sim_list
    

In [18]:
# 4개년도 모델을 학습하고 저장합니다
models = []
for i in tqdm(range(4)):
    model = doc2vec_train(know_train[i])
    models.append(model)

100%|██████████| 4/4 [05:49<00:00, 87.37s/it] 


In [ ]:
# 4개년도 데이터로 예측합니다
doc2vec_preds = []
for i in tqdm(range(4)):
    pred, _ = doc2vec_pred(know_train[i], know_test[i], models[i])
    doc2vec_preds.extend(pred)

In [29]:
submission = pd.read_csv('../data_0103/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = doc2vec_preds

submission.to_csv('doc2vec_data0116.csv', index=False)

# Result Preview
> 성능이 안좋아서 어떻게 예측이 진행됐는지 눈으로 확인함

In [19]:
sims = []
for i in tqdm(range(4)):
    pred, sim = doc2vec_pred(know_train[i], know_test[i], models[i])
    sims.append(sim)

100%|██████████| 4/4 [03:25<00:00, 51.44s/it]


In [20]:
# similarity가 다 너무 높게 나온다
# 뭔가 이상하다
for i in range(4):
    print(pd.Series(sims[i]).describe())
    print()

count    9486.000000
mean        0.118696
std         0.089266
min         0.050000
25%         0.090000
50%         0.100000
75%         0.120000
max         0.830000
dtype: float64

count    9069.000000
mean        0.136223
std         0.143112
min         0.050000
25%         0.090000
50%         0.100000
75%         0.120000
max         0.910000
dtype: float64

count    8554.000000
mean        0.136568
std         0.145903
min         0.060000
25%         0.090000
50%         0.100000
75%         0.120000
max         0.880000
dtype: float64

count    8122.000000
mean        0.103090
std         0.042041
min         0.050000
25%         0.090000
50%         0.100000
75%         0.110000
max         0.850000
dtype: float64



In [21]:
train_data = know_train[0]
test_data = know_test[0]

Test_documents = [doc for doc in test_data['text_response']]
    
pred_list = []
sim_list = []
verbose_df = pd.DataFrame(index=range(len(Test_documents)))
for i,text in enumerate(Test_documents):
    text_split = text.split(' ')
    inferred_v = models[0].infer_vector(text_split)
    most_similar_docs = models[0].docvecs.most_similar(positive=[inferred_v], topn=1)
    
    top1_train_idx = int(most_similar_docs[0][0])
    top1_sim = round(float(most_similar_docs[0][1]),2)
    sim_list.append(top1_sim)
    top1_train_desc = list(train_data.loc[train_data['idx']==top1_train_idx]['text_response'])[0]
    
    pred = list(train_data.loc[train_data['idx']==top1_train_idx]['knowcode'])[0]
    pred_list.append(pred)
    
    test_idx = test_data.loc[i,'idx']
    
    verbose_df.loc[i,'test_idx'] = test_idx
    verbose_df.loc[i,'sim_train_idx'] = top1_train_idx
    verbose_df.loc[i,'sim_train_desc'] = top1_train_desc
    verbose_df.loc[i,'sim_test_desc'] = test_data.loc[i,'text_response']
    verbose_df.loc[i,'similarity'] = top1_sim

In [22]:
# Doc2vec이 제대로 이루어지지 않음을 볼 수 있다. 해당 모델은 쓰기 힘들 것으로 생각된다.
verbose_df.head(49)

,test_idx,sim_train_idx,sim_train_desc,sim_test_desc,similarity
0,0.0,7617.0,"해외정서및 기본소양교육 변화 사무관, 서기관, 대사 정외",대체 업무 컴퓨터 비서학,0.10
1,1.0,4793.0,영상 방송에 관한 업무 미디어 시대 도래 카메라 컴퓨터 플래시 감독님 방송미디어,품질관리기사 실무교육 제조업 생각 때문 제품 검사 시스템 엑셀 농화학,0.11
2,2.0,1938.0,교육전도사 과정 교인 감소 요인 인구 감소 성경책 주석 십자가 자영업 신학,기획력 현장경험 미디어 증가 오디션 편집 프로그램 홀로 방송 진행 장비 1인 미디어...,0.09
3,3.0,8790.0,치과의면허 고령화 노련 증가 보철 인플란트 분야 수요 증가 치과 광중 압기 미세 현...,선호 직업 컴퓨터 화학,0.06
4,4.0,3111.0,운전면허증 대중교통 이용 때문 자동차 카카오 택시 어플 택시기사 자영업 기계과,잡지 정보 뉴스 고갈 때문 유지 뉴스 원고 편집기자 사진작가 리포터 광고홍보,0.11
5,5.0,6459.0,디지털포렌식 CCFP 정보보안기사 정보과자격증 필요 사이버 범죄를 수사라는 기법교육...,코드 익히는 멜로디 연주법 리듬주법 저출산 자동화 변화 때문 피크 엠프 컴퓨터 악보...,0.11
6,6.0,6707.0,회계 무량 전산 회계 프로그램 회계,보육교사자격증 여성 사회 진출 증가 교재 도구 놀이 도구 선생님 피아노강사 아동복지보육,0.10
7,7.0,4504.0,치기공사 수요 처가 보험 석고 모형 마스크 프레 사무직 치위생사 치기공학,국제법이해 행정법 정보분석 대체 때문 한글 작성 엑셀 전자전기학과,0.12
8,8.0,5683.0,바둑기사 자격증 한국기원의 기사선발전 통해 입단 기원들 간의 교류 온라인게임 바둑 ...,내부 규정 이해 실제 사계교육 과거 비교 변화 때문 보안 장비 선생님 교정보호학,0.11
9,9.0,4676.0,실무와 관련된 시험에 대한 교육 매출 일지 포화상태 발열 장비 원소 석기 엑셀 화학과,자동차 운전사 증가 대중교통 이용자 감소 버스기사 개인택시기사 상업고,0.10
